# Prompt Chaining Examples

This notebook collects four practical examples of chaining multiple prompts or steps together to accomplish a complex task.

## Example 1: Research-Paper Digest → Quiz Builder
This chain summarises a long paper, generates MCQs for its contributions, and verifies coverage.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

summary_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\n{paper}\n—\nExtract a crisp abstract (≤150 words) and list the 5 main contributions.\n'),
    output_key='summary'
)

mcq_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nContributions:\n{summary}\n—\nWrite one challenging MCQ for EACH contribution. Indicate the correct option.\n'),
    output_key='mcqs'
)

review_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nMCQs produced:\n{mcqs}\n—\nDo we have exactly 5 questions, one per contribution?\n- If yes, reply “✅ ready”.\n- If any contribution lacks a question, add it and then reply “✅ ready”.\n'),
    output_key='final_mcqs'
)

paper_chain = SequentialChain(
    chains=[summary_chain, mcq_chain, review_chain],
    input_variables=['paper'],
    output_variables=['final_mcqs'],
    verbose=True
)

# Example usage (requires OPENAI_API_KEY)
# paper_text = open('some_paper.txt').read()
# result = paper_chain.run(paper=paper_text)


## Example 2: Raw Marks → Performance Dashboard → Parent Emails
This chain analyses marks, drafts a short dashboard snippet, and prepares bilingual emails for at-risk students.

In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

df = pd.read_csv('ds_marks.csv')

analyze_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template("\nYou are a data-analytics tutor.\nGiven this JSON list of dicts [{name, roll, marks}] (max 50),\nreturn JSON:\n  {{ 'safe_count': int,\n     'risk_count': int,\n     'at_risk': [{{name, roll, marks}}] }}\nStudents scoring <40 % are at_risk.\n\nDATA:\n{data}\n"),
    output_key='analysis'
)

email_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nCreate a personalised email for EACH entry in {{at_risk_json}}.\nFormat:\n### <Roll> – <Name>\nSubject: Performance Alert – DS\nBody (80-120 words):\n  • Explain shortfall\n  • Invite to remedial on Friday 4 pm\n  • Close with one Telugu sentence encouraging them\nReturn all emails in plain text.\n'),
    output_key='emails'
)

hod_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template("\nSafe: {{safe}}; At-Risk: {{risk}}\nWrite a ≤120-word summary for HoD, mentioning remedial plan.\nAppend the full email block below a 'Drafts' heading.\nEmails:\n{{emails}}\n"),
    output_key='hod_report'
)

marks_chain = SequentialChain(
    chains=[analyze_chain, email_chain, hod_chain],
    input_variables=['data'],
    output_variables=['hod_report'],
    verbose=True
)

# Example usage (requires OPENAI_API_KEY)
# report = marks_chain.run(data=df.to_dict(orient='records'))


## Example 3: Bug Report → Root-Cause Hypothesis → Patch Suggestion
This chain extracts traces from logs, reasons about the root cause, proposes a patch, and checks if security-sensitive code paths are touched.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

extract_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\n{log}\n—\nPull out stack traces and the last 20 lines before the crash.\n'),
    output_key='extract'
)

cause_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nTrace + context:\n{extract}\n—\nSuggest the most likely root cause in ≤50 words and list any missing information you’d need to confirm.\n'),
    output_key='cause'
)

patch_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nHypothesis:\n{cause}\n—\nPropose a code-level fix (language: Java, Spring Boot). Show only the diff.\n'),
    output_key='patch'
)

safety_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template("\nProposed diff:\n{patch}\n—\nDoes this patch alter authentication or encryption code paths?\n• If yes, flag 'SECURITY REVIEW NEEDED'.\n• If no, flag 'OK'.\n"),
    output_key='status'
)

bug_chain = SequentialChain(
    chains=[extract_chain, cause_chain, patch_chain, safety_chain],
    input_variables=['log'],
    output_variables=['status'],
    verbose=True
)

# Example usage (requires OPENAI_API_KEY)
# status = bug_chain.run(log=open('server.log').read())


## Example 4: Lecture Notes → Learning Objectives → MCQs → QA Check
This chain extracts learning objectives, generates MCQs, and repeats until each objective has a matching question.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain, SequentialChainException

with open('embedded_systems_unit3.txt') as f:
    lecture_text = f.read()

obj_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template('\nTEXT:\n{para}\n----\nProduce 3–5 precise learning objectives in bullet form.\n'),
    output_key='objectives'
)

mcq_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template("\nObjectives:\n{objectives}\n----\nFor EACH objective generate exactly one 4-option MCQ.\nReturn JSON list: [{{'objective': str, 'mcq': str, 'answer': 'A/B/C/D', 'bloom': 'L1-L6'}}]\n"),
    output_key='mcqs'
)

check_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=PromptTemplate.from_template("\nObjectives:\n{objectives}\n\nMCQs:\n{mcqs}\n----\nVerify: Is count(objectives)==count(mcqs) and each objective appears verbatim in one MCQ?\nIf any mismatch, reply 'FIX' and list missing objectives; else reply 'PASS'.\n"),
    output_key='status'
)

def run_until_pass(para: str, max_loops=3):
    for _ in range(max_loops):
        objectives = obj_chain.run(para=para)
        mcqs = mcq_chain.run(objectives=objectives)
        status = check_chain.run(objectives=objectives, mcqs=mcqs)
        if 'PASS' in status:
            return mcqs
    raise SequentialChainException('QA never passed; try again.')

# Example usage (requires OPENAI_API_KEY)
# mcq_block = run_until_pass(lecture_text)
